# Configuration de Cuda dans Google Colab

In [1]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-dwfvsthc
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-dwfvsthc
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=84ca9f2cbf958736c254af699e4d82e4f130eca69c35e1fe105d1cd0632f6d9f
  Stored in directory: /tmp/pip-ephem-wheel-cache-4wps55m3/wheels/db/c1/1f/a2bb07bbb4a1ce3c43921252aeafaa6205f08637e292496f04
Successfully built NVCCPlugin


## On vérifie que l'on est bien connecté au GPU

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar 19 17:44:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Chargement du plugin nvcc permettant de compiler/executer les programmes Cuda

In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


## Un makefile est déjà à votre disposition pour compiler les programme du TP


### 1. Executez la cellule du Makefile

Le makefile a été modifié pour les programmes puisse s'exécuter avec la GPU premium.



In [5]:
%%writefile Makefile
# Change the example variable to build a different source module (e.g. EXAMPLE=exercice01)
EXAMPLE=program

# Makefile variables 
# Add extra targets to OBJ with space separator e.g. If there is as source file random.c then add random.o to OBJ)
# Add any additional dependancies (header files) to DEPS. e.g. if there is aheader file random.h required by your source modules then add this to DEPS.
CC=gcc
CFLAGS= -O3 -Wextra -fopenmp
NVCC=nvcc
NVCC_FLAGS= -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_87,code=sm_87
OBJ=$(EXAMPLE).o
DEPS=

# Build rule for object files ($@ is left hand side of rule, $< is first item from the right hand side of rule)
%.o : %.cu $(DEPS)
	$(NVCC) -c -o $@ $< $(NVCC_FLAGS) $(addprefix -Xcompiler ,$(CCFLAGS))

# Make example ($^ is all items from right hand side of the rule)
$(EXAMPLE) : $(OBJ)
	$(NVCC) -o $@ $^ $(NVCC_FLAGS) $(addprefix -Xcompiler ,$(CCFLAGS))

# PHONY prevents make from doing something with a filename called clean
.PHONY : clean
clean:
	rm -rf $(EXAMPLE) $(OBJ)

Writing Makefile


# TP3


## Fonctions utilitaires

utils.h est un header contenant des fonctions utilitaires qui seront utilisés par nos programmes

In [6]:
%%writefile utils.h
#ifndef __UTILS_H__
#define __UTILS_H__
#include <stdio.h>

static void HandleError( cudaError_t err,
                         const char *file,
                         int line ) {
    if (err != cudaSuccess) {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}
#define HANDLE_ERROR( err ) (HandleError( err, __FILE__, __LINE__ ))


#define HANDLE_NULL( a ) {if (a == NULL) { \
                            printf( "Host memory failed in %s at line %d\n", \
                                    __FILE__, __LINE__ ); \
                            exit( EXIT_FAILURE );}}

template< typename T >
void swap( T& a, T& b ) {
    T t = a;
    a = b;
    b = t;
}

struct event_pair
{
  cudaEvent_t start;
  cudaEvent_t end;
};

inline void start_timer(event_pair * p)
{
  cudaEventCreate(&p->start);
  cudaEventCreate(&p->end);
  cudaEventRecord(p->start, 0);
}


inline void stop_timer(event_pair * p, char * kernel_name)
{
  cudaEventRecord(p->end, 0);
  cudaEventSynchronize(p->end);
  
  float elapsed_time;
  cudaEventElapsedTime(&elapsed_time, p->start, p->end);
  printf("%s took %.4f ms\n",kernel_name, elapsed_time);
  cudaEventDestroy(p->start);
  cudaEventDestroy(p->end);
}

void* big_random_block( int size ) {
    unsigned char *data = (unsigned char*)malloc( size );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}

int* big_random_block_int( int size ) {
    int *data = (int*)malloc( size * sizeof(int) );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}


// a place for common kernels - starts here

__device__ unsigned char value( float n1, float n2, int hue ) {
    if (hue > 360)      hue -= 360;
    else if (hue < 0)   hue += 360;

    if (hue < 60)
        return (unsigned char)(255 * (n1 + (n2-n1)*hue/60));
    if (hue < 180)
        return (unsigned char)(255 * n2);
    if (hue < 240)
        return (unsigned char)(255 * (n1 + (n2-n1)*(240-hue)/60));
    return (unsigned char)(255 * n1);
}

__global__ void float_to_color( unsigned char *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset*4 + 0] = value( m1, m2, h+120 );
    optr[offset*4 + 1] = value( m1, m2, h );
    optr[offset*4 + 2] = value( m1, m2, h -120 );
    optr[offset*4 + 3] = 255;
}

__global__ void float_to_color( uchar4 *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset].x = value( m1, m2, h+120 );
    optr[offset].y = value( m1, m2, h );
    optr[offset].z = value( m1, m2, h -120 );
    optr[offset].w = 255;
}


#if _WIN32
    //Windows threads.
    #include <windows.h>

    typedef HANDLE CUTThread;
    typedef unsigned (WINAPI *CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC unsigned WINAPI
    #define  CUT_THREADEND return 0

#else
    //POSIX threads.
    #include <pthread.h>

    typedef pthread_t CUTThread;
    typedef void *(*CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC void
    #define  CUT_THREADEND
#endif

//Create thread.
CUTThread start_thread( CUT_THREADROUTINE, void *data );

//Wait for thread to finish.
void end_thread( CUTThread thread );

//Destroy thread.
void destroy_thread( CUTThread thread );

//Wait for multiple threads.
void wait_for_threads( const CUTThread *threads, int num );

#if _WIN32
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void *data){
        return CreateThread(NULL, 0, (LPTHREAD_START_ROUTINE)func, data, 0, NULL);
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        WaitForSingleObject(thread, INFINITE);
        CloseHandle(thread);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        TerminateThread(thread, 0);
        CloseHandle(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        WaitForMultipleObjects(num, threads, true, INFINITE);

        for(int i = 0; i < num; i++)
            CloseHandle(threads[i]);
    }

#else
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void * data){
        pthread_t thread;
        pthread_create(&thread, NULL, func, data);
        return thread;
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        pthread_join(thread, NULL);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        pthread_cancel(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        for(int i = 0; i < num; i++)
            end_thread( threads[i] );
    }

#endif




#endif  // __UTILS_H__

Writing utils.h


## 1. Convolution simple (1D)

La convolution est une opération permettant d'alterer une image en faisant glisser un filtre sur cette image. Cela peut permettre d'ajouter du flou sur l'image, de detecter le contours de certains objets d'un image et bien d'autres.
(Cf images https://github.com/GPUModule/cuda-TP3/blob/main/README.md)

Le but de cet exercice est de programmer une convolution simple, ou l'on fait glisser un filtre de taille 3 (tableau de taille 3) sur un vecteur a de taille N

Le vecteur c contenant le résultat de la convolution n'a pas la même taille que le que vecteur a d'origine Pour trouver la taille du tableau résultant, il suffit de faire **n_c = N - f + 1**.

Une fonction **random_floats** vous est fournit permettant d'initialiser un tableau de taille N.

### Complétez la fonction main.
#### 1.1 Allouez la mémoire hôte à l'aide de malloc
#### 1.2 Allouez la mémoire GPU à l'aide de cudaMalloc
#### 1.3 Copiez les vecteurs a et b dans la mémoire GPU (respectivement d_a et d_b) 
#### 1.4 Copiez la mémoire du de d_c dans la mémoire hôte (c)
#### 1.5 LIbéré la mémoire.


### Convolution simple
#### 1.6 Completez le kernel simple_convolution_1D_kernel
- Créez une variable temporaire stockant le résultat de la convolution pour chaque thread.
- Chaque thread multipliera chaque élément de f avec les éléments correspondant de a, et sommera le résultat de chaque multiplication. La résultat de la somme sera stocké dans c.

#### 1.7 Executez le programme 
Le programme doit afficher un tableau a de 6 éléments, un filtre filter de 3 éléments et le résultat c de 4 éléments.

On commence avec un tableau de taille 6 afin de valider rapidement le résultat de la convolution.

Une fois le résultat validé, changez les paramètres suivant :
```
	dim3 blocksPerGrid(1) --> dim3 blocksPerGrid((N + THREADS_PER_BLOCK -1)/THREADS_PER_BLOCK); 
```

```
  #define N 6 --> #define N 2048
  #define THREADS_PER_BLOCK 6 --> #define THREADS_PER_BLOCK 256
```
  
Commentez l'affichage des vecteurs:
```
print_array(a, N, "a"); --> print_array(a, N, "a"); 
print_array(filter, f, "filter"); --> print_array(filter, f, "filter");

print_array(c, n_c, "c"); --> print_array(c, n_c, "c");
```
### Convolution simple dans la mémoire partagée.

#### 1.8 Dans la fonction main, écrivez les instructions nécessaire à la création d'un vecteur cs et d'un vecteur d_cs de même dimension. cs récupérera le résultat de la convolution faite en utilisant la mémoire partagée, stocké dans d_cs.

#### 1.9 Complétez le kernel shared_convolution_1D
- Creer un vecteur s_data en mémoire partagée d'une taille égale à la dimension d'un bloc.
- Initialisez s_data avec les éléments du vecteur a. Utilisez l'index global des threads pour récupérer les bon éléments du vecteur a.
- Synchroniser les threads (\_\_syncthreads()) pour qu'ils attendent que s_data soit bien initialisé. 

Modifié le code afin de pouvoir utiliser cette mémoire partagée. 

### 1.10 Decommentez les instructions suivante :

```
//errors = validate(c, cs, n_c);
//printf("CUDA GPU result has %d errors.\n", errors);
```

Cela de s'assurer que le résultat de la convolution simple utilisant la mémoire globale et de la convolution simple utilisant la mémoire partagée sont identiques.

Faites attentions aux limites du tableau s_data, chaque block creer un tableau s_data de taille le nombre de threads de ce block. il récupère la sous-partie correspondante du tableau a dans s_data et réalise ensuite la convolution. Les threads d'un block n'ont pas accès au tableau s_data d'autres blocks.


In [103]:
%%writefile program.cu
#include <stdlib.h>
#include <stdio.h>
#include <math.h>
#include <cuda_runtime.h>
#include "utils.h"


#define N 2048
#define THREADS_PER_BLOCK 256
#define SQRT_THREADS_PER_BLOCK sqrt(THREADS_PER_BLOCK)

void random_floats(float *a, int n);
void print_array(float *a, int n, char *name);
int validate(float *a, float *ref, int n);

__global__ void simple_convolution1D_kernel(float* c, float* a, float* filter, int f, int n) {// N = 6 -> 0.03 ms. N = 
		int row = blockIdx.x * blockDim.x + threadIdx.x;

		if(row < n - f + 1){
			float Cvalue = 0;
			for(int i = 0; i < f; i++){
						Cvalue += a[row + i] * filter[i];
			}
			c[row] = Cvalue;
		}
		
}

__global__ void shared_convolution1D_kernel(float* c, float* a, float* filter, int f, int n) {
		
		__shared__ float s_data[THREADS_PER_BLOCK + 2]; // +2 pour les variables qui ne seront pas accessible
		int i = blockIdx.x * blockDim.x + threadIdx.x;
		int s_data_idx = threadIdx.x +1;
		
		s_data[s_data_idx] = (i < n) ? a[i] : 0.0f;
		s_data[blockDim.x + 1] = (blockIdx.x < N - 1) ? a[blockDim.x * blockIdx.x + blockDim.x] : 0.0f;
		s_data[blockDim.x + 2] = (blockIdx.x < N - 1) ? a[blockDim.x * blockIdx.x + blockDim.x + 1] : 0.0f;
		s_data[0] = (blockIdx.x > 0) ?  a[i - 1] : 0.0f;		

		__syncthreads();

		float sum = 0.0f;
		for (int j = 0; j < f; j++) {
			sum += filter[j] * s_data[s_data_idx + j];
		}
		c[i] = sum;
}


int main(void) {
	srand( time( NULL ) );

	float *a, *filter, *c, *cs;
	float *d_a, *d_filter, *d_c, *d_cs;
	int errors;
	
	int f = 3;
	int n_c = N - f + 1 ;

	event_pair timer;

  // 1.1. Allocation de la mémoire hôte
	unsigned int filter_size = f * sizeof(float);
	unsigned int size = N * sizeof(float);
	unsigned int c_size = n_c * sizeof(float);
	a = (float*) malloc(size);
	filter = (float*) malloc(filter_size);
	c = (float*) malloc(c_size);
	cs = (float*) malloc(c_size);

	// 1.2. Allocation de la mémoire GPU
	// A completer
	HANDLE_ERROR( cudaMalloc(&d_a,  size ) );
  HANDLE_ERROR( cudaMalloc(&d_filter, filter_size ) );
  HANDLE_ERROR( cudaMalloc(&d_c, c_size ) );
	HANDLE_ERROR( cudaMalloc(&d_cs, c_size) );

	random_floats(a, N);
	random_floats(filter, f);
	//print_array(a, N, "a");
	//print_array(filter, f, "filter");

	// 1.3. Copie de la mémoire Hôte vers la mémoire GPU
	// A completer
	HANDLE_ERROR( cudaMemcpy( d_a, a, size, cudaMemcpyHostToDevice ) );
  HANDLE_ERROR( cudaMemcpy( d_filter, filter, filter_size, cudaMemcpyHostToDevice) ); 

	// Lancement du kernel.
	dim3 blocksPerGrid((N + THREADS_PER_BLOCK -1)/THREADS_PER_BLOCK);
	dim3 threadsPerBlock(THREADS_PER_BLOCK);
	//start_timer(&timer);
	simple_convolution1D_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_c, d_a, d_filter, f, N);
	//stop_timer(&timer,"Convolution 1D sur GPU");
	
	// 1.4. Copie de la mémoire GPU --> CPU
	HANDLE_ERROR(cudaMemcpy(c, d_c ,c_size, cudaMemcpyDeviceToHost) );


	//print_array(c, n_c, "c");
	
	start_timer(&timer);
	shared_convolution1D_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_cs, d_a, d_filter, f, N);
	stop_timer(&timer,"Convolution 1D shared sur GPU");
	

	HANDLE_ERROR(cudaMemcpy(cs, d_cs ,c_size, cudaMemcpyDeviceToHost));	

	// validate
	errors = validate(c, cs, n_c);
	printf("CUDA GPU result has %d errors.\n", errors);

	// 1.5. Libérer la mémoire.

	free(a);
	free(filter);
	free(c);
	cudaFree(d_c);
	cudaFree(d_a);
	cudaFree(d_filter);
  
	return 0;
}

void random_floats(float *a, int n)
{
	for (unsigned int i = 0; i < n; i++){
			a[i] = (float)(rand() % 101);
	}
}

void print_array(float *a, int n, char*name){

	printf("%s : [ ",name);
	for (unsigned int i = 0; i < n; i++){
			printf("%.4f ",a[i]);
	}
	printf("]\n");
}

int validate(float *a, float *ref, int n){
	int errors = 0;
	for (unsigned int i = 0; i < n; i++){
		if (a[i] != ref[i]){
			errors++;
			fprintf(stderr, "ERROR at index %d: GPU result %f does not match CPU value of %f\n", i, a[i], ref[i]);
		}
	}

	return errors;
}

Overwriting program.cu


In [104]:
!make

nvcc -c -o program.o program.cu -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_87,code=sm_87 
program.cu(103): warning #2464-D: conversion from a string literal to "char *" is deprecated

program.cu(103): warning #2464-D: conversion from a string literal to "char *" is deprecated

program.cu(103): warning #2464-D: conversion from a string literal to "char *" is deprecated

program.cu(103): warning #2464-D: conversion from a string literal to "char *" is deprecated

program.cu(103): warning #2464-D: conversion from a string literal to "char *" is deprecated

nvcc -o program program.o -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_87,code=sm_87 


In [105]:
!./program

Convolution 1D shared sur GPU took 0.0105 ms
CUDA GPU result has 0 errors.
